In [ ]:
from selenium import webdriver
import RPi.GPIO as GPIO
import time
from rpi_ws281x import *
import argparse

LED_COUNT      = 1
LED_PIN        = 18

strip = PixelStrip(LED_COUNT, LED_PIN)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')

driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', options=options)
driver.get('http://192.168.137.127')

def colorWipe(strip, color, wait_ms=50):
    """Wipe color across display a pixel at a time."""
    for i in range(strip.numPixels()):
        strip.setPixelColor(i, color)
        strip.show()
        time.sleep(1)
        
while True:
    time.sleep(10)
    fire = driver.find_element_by_css_selector('body > span.fire > em').text
    gas = driver.find_element_by_css_selector('body > span.gas > em').text
    print(fire)
    print(gas)
    ifire = float(fire)
    igas = float(gas)
    if ifire == 0 and igas >= 300:
        strip.begin()
        buzzer = 23
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(buzzer, GPIO.OUT)
        GPIO.setwarnings(False)
        list = [261, 294, 329, 349, 392, 440, 493 ,523]
        while True:
            try:
                p = GPIO.PWM(buzzer, 100) 
                p.start(100)
                p.ChangeDutyCycle(90)
                for i in range(8):
                    p.ChangeFrequency(list[i])
                    time.sleep(1)
                p.stop()
                colorWipe(strip, Color(255, 0, 0))
                colorWipe(strip, Color(0, 255, 0))
                colorWipe(strip, Color(0, 0, 255))
            except:pass
    else:pass